In [53]:
import pandas as pd
import numpy as np

file='HNDB Progress Notes Processed.csv'
df=pd.read_csv(file)
print(df.columns)
cols=['mrn','date_of_service_dttm','filtered_HPI']
df=df[cols]
df['filtered_HPI']=df['filtered_HPI'].fillna('NO NEW INFORMATION')
print(df.head())

Index(['date_of_service_dttm', 'mrn', 'note_narr', 'HPI', 'First_Sent',
       'TNM_stage', 'T', 'N', 'M', 'Cancer_stage', 'last_HPI',
       'changes_in_HPI', 'filtered_HPI'],
      dtype='object')
       mrn date_of_service_dttm  \
0  3486779           1978-01-01   
1  3486779           2012-11-01   
2  3486779           2013-02-01   
3  3486779           2014-09-01   
4  3486779           2014-10-01   

                                        filtered_HPI  
0                              Quit date: 1/1/1978.   
1           -- November 2012 noticed in right neck.   
2  -- Jan--Feb 2013 cisplatin based chemoradiatio...  
3  -- 9/2014 Noticed pain right hip -- noticed to...  
4  -- Oct 2014 surgery with rod placement per Dr....  


In [54]:
mrns=df.mrn.unique()
ptx_info=[]
for mrn in mrns:
    sub_df=df[df['mrn']==mrn]
    ptx_info.append(sub_df)
    
def collapse_HPI(df):
    paragraph=''
    for index,row in df.iterrows():
        paragraph=paragraph+row['date_of_service_dttm']+':\t'+row['filtered_HPI']+'|'
    return paragraph
data=[]
for index,ptx in enumerate(ptx_info):
    datum=[ptx.iloc[0]['mrn'],collapse_HPI(ptx)]
    data.append(datum)
df_nar=pd.DataFrame(data=data,columns=['mrn','narrative'])

In [55]:
print(df_nar.iloc[0]['narrative'])

import scispacy
import spacy
import en_core_sci_md   #The model we are going to use
import re

nlp = spacy.load("en_core_sci_md")
counter=0
def get_Diagnosis(text):
    doc = nlp(text)
    pattern=r'[Cc]ancer(?!\s)|[Aa]denoma|[Cc]arcinoma|[Aa]denocarcinoma|HNC|SCC|[Tt]umor(?!\s)'
    diags=[]
    
    for ent in doc.ents:
        if re.search(pattern,ent.text):
            diags.append(ent.text)
    global counter
    counter=counter+1
    print(counter)
    return diags

df_nar['Dx']=df_nar['narrative'].apply(get_Diagnosis)
print (df_nar.iloc[0]['Dx'])

1978-01-01:	Quit date: 1/1/1978. |2012-11-01:	-- November 2012 noticed in right neck. |2013-02-01:	-- Jan--Feb 2013 cisplatin based chemoradiation -- cisplatin stopped early due to kidney problems -- some residual kidney impairment. |2014-09-01:	-- 9/2014 Noticed pain right hip -- noticed to have bone metastasis with risk of fractured. |2014-10-01:	-- Oct 2014 surgery with rod placement per Dr. Peabody. |2015-08-01:	-- 8/2015 2nd PET scan -- activity in the femur + spots in the lung (too small to biopsy) + recurrent right hip pain. |2015-09-09:	-- Started carboplatin/paclitxel + Cetuximab (Extreme) 9/9/2015 -- Had PORT place. |2016-05-16:	-- PET on 5/16/2016 with PD and new lesions. |2016-05-25:	Lewis Hoose is a 67Yrs male here for follow-up of metastatic HNC here for PD-1/Lirilumumab trial (anti-KIR). |2016-06-22:	-- 6/22/16: Started Nivo/Kir study. |2016-06-29:	Lewis Hoose is a 67Yrs male with metastatic HNC here for PD-1/Lirilumumab trial (anti-KIR). He presents for C1D1. States he 

In [56]:
def remove_dups(l):
    mylist = list(dict.fromkeys(l))
    
    sorted_list = sorted(mylist, key=len)
    
    return sorted_list

df_nar['Dx_']=df_nar['Dx'].apply(remove_dups)


for index,row in df_nar.iterrows():
    print(row['Dx_'],'\n')

df_export=df_nar[['mrn','narrative','Dx_']]
print(df_export.head(30))
df_export.to_csv('RM_narrative.csv',index=False)

['HNC', 'HNSCC', 'cancer', 'metastatic HNC', 'metastatic HNSCC'] 

['SCC', 'SCCA', 'no tumor', 'head neck tumor', 'shrinking tumor', 'head-neck cancer', 'esophageal cancer', 'Esophageal cancer', 'N2C supraglottic laryngeal cancer'] 

['SCCA', 'T2N0M0 SCCA', 'breast cancer', 'right breast cancer', 'squamous cell cancer', 'squamous cell carcinoma', 'left oral tongue cancer', 'primary squamous cell carcinoma'] 

['SCC', 'HNC', 'SCCHN', 'no tumor', 'carcinoma', 'recurrent cancer', 'in situ carcinoma', 'oral tongue carcinoma', 'squamous cell carcinoma', 'right paratracheal tumor', 'right tonsil squamous cell carcinoma'] 

['SCC', 'cancer', 'HN cancer', 'tongue cancer', 'prostate cancer', 'Prostate cancer', 'metastatic cancer', 'no viable carcinoma', 'HPV+ right bot cancer', 'HPV+ right BOT cancer', 'metastatic prostate cancer', 'PSA/testosterone      Cancer', 'hormone-sensitive prostate cancer'] 

['SCC', 'Tongue cancer', 'oral tongue cancer'] 

['Extensive tumor', 'soft palate carcinoma', 